In [1]:
from tensorflow.keras.layers import Dense, Flatten, Input 
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.applications.xception import Xception, preprocess_input
from glob import glob
import numpy as np
import matplotlib.pyplot as plt


In [2]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='JTI-ESd9rcAvtdIZMmTR67PUxIqz28bHCbkEDEhwjV3d',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'plantseedlingmodeldeployment-donotdelete-pr-tr0fzub0dtb924'
object_key = 'dataset.zip'

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [3]:
ls -1

test/
train/


In [4]:
from io import BytesIO
import zipfile
unzip=zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths=unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [5]:
ls -1

test/
train/


In [6]:
pwd

'/home/wsuser/work'

In [7]:
ls

test/  train/


In [8]:
cd /home/wsuser/work/train

/home/wsuser/work/train


In [9]:
cd /home/wsuser/work/test

/home/wsuser/work/test


In [10]:
imageSize = [224, 224]

trainPath = r"/home/wsuser/work/train"

testPath = r"/home/wsuser/work/test"


In [11]:
xception = Xception(input_shape=imageSize + [3], weights='imagenet',include_top=False)

In [12]:
# don't train existing weights
for layer in xception.layers:
  layer.trainable = False

In [13]:
# our layers - you can add more if you want
x = Flatten()(xception.output)

In [14]:
prediction = Dense(12, activation='softmax')(x)

In [15]:
# create a model object
model = Model(inputs=xception.input, outputs=prediction)

In [16]:
# view the structure of the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 111, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 111, 111, 32  128        ['block1_conv1[0][0]']           
 ation)                         )                                                             

In [17]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)


In [18]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [19]:
training_set = train_datagen.flow_from_directory(trainPath,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory(testPath,
                                            target_size = (224,224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 3793 images belonging to 12 classes.
Found 957 images belonging to 12 classes.


In [20]:
# fit the model
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=50,
  steps_per_epoch=len(training_set)//5,
  validation_steps=len(test_set)//5
)

/tmp/wsuser/ipykernel_1324/2688581469.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(


Epoch 1/50
23/23 [==============================] - 111s 5s/step - loss: 5.5699 - accuracy: 0.3410 - val_loss: 4.7578 - val_accuracy: 0.4792
Epoch 2/50
23/23 [==============================] - 112s 5s/step - loss: 2.3548 - accuracy: 0.6291 - val_loss: 2.4013 - val_accuracy: 0.5990
Epoch 4/50
23/23 [==============================] - 114s 5s/step - loss: 2.4624 - accuracy: 0.6522 - val_loss: 1.4552 - val_accuracy: 0.7292
Epoch 5/50
23/23 [==============================] - 111s 5s/step - loss: 2.2547 - accuracy: 0.6590 - val_loss: 2.0299 - val_accuracy: 0.6823
Epoch 6/50
23/23 [==============================] - 112s 5s/step - loss: 2.0528 - accuracy: 0.6997 - val_loss: 2.0558 - val_accuracy: 0.7083
Epoch 7/50
23/23 [==============================] - 110s 5s/step - loss: 2.0155 - accuracy: 0.6984 - val_loss: 2.5904 - val_accuracy: 0.6406
Epoch 8/50
23/23 [==============================] - 113s 5s/step - loss: 1.7546 - accuracy: 0.7405 - val_loss: 2.5224 - val_accuracy: 0.6510
Epoch 9/50
23

In [21]:
model.save('seedling_exception.h5')

/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


In [25]:
!tar -zcvf seedling_exception.tgz seedling_exception.h5

seedling_exception.h5


In [26]:
!pip install watson-machine-learning-client

     |████████████████████████████████| 538 kB 16.7 MB/s eta 0:00:01


In [27]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {
                    "url": "https://us-south.ml.cloud.ibm.com",
                    "apikey": "0lJUIRhoIeMyC5PSBziSukzetcr7wFV5xPid11vhqcob"
                  }
client = APIClient(wml_credentials)

In [28]:
client = APIClient(wml_credentials)

In [29]:
def guid_from_space_name(client, Seed):
    space = client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']["name"] ==Seed)['metadata']['id'])

In [30]:
space_uid = guid_from_space_name(client, 'Seed')
print("SPACE UID = " + space_uid )

SPACE UID = 5059f357-1e4d-4b16-90ce-f26f0bbf8318


In [31]:
client.set.default_space(space_uid)

'SUCCESS'

In [32]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
runtime-22.1-py3.9             12b83a1

In [47]:
software_spec_uid = client.software_specifications.get_uid_by_name("runtime-22.1-py3.9")
software_spec_uid

'12b83a17-24d8-5082-900f-0ab31fbfd3cb'

In [48]:
ls

 Black-grass/        'Fat Hen'/               seedling_exception.tgz
 Charlock/           'Loose Silky-bent'/     'Shepherds Purse'/
 Cleavers/            Maize/                 'Small-flowered Cranesbill'/
'Common Chickweed'/  'Scentless Mayweed'/    'Sugar beet'/
'Common wheat'/       seedling_exception.h5


In [35]:
import tensorflow
tensorflow.__version__

'2.7.2'

In [49]:
model_details=client.repository.store_model(model='seedling_exception.tgz',
            meta_props={
                        client.repository.ModelMetaNames.NAME:"CNN",
                        client.repository.ModelMetaNames.TYPE:'tensorflow_2.7',  
                        client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid
})

In [50]:
model_id=client.repository.get_model_id(model_details)

In [51]:
model_id

'55d3440d-7d62-4438-bf13-ea167a956e4f'

In [52]:
ls

 Black-grass/        'Fat Hen'/               seedling_exception.tgz
 Charlock/           'Loose Silky-bent'/     'Shepherds Purse'/
 Cleavers/            Maize/                 'Small-flowered Cranesbill'/
'Common Chickweed'/  'Scentless Mayweed'/    'Sugar beet'/
'Common wheat'/       seedling_exception.h5


In [53]:
client.repository.download('55d3440d-7d62-4438-bf13-ea167a956e4f','seedling_exceptionml.tgz')

Successfully saved model content to file: 'seedling_exceptionml.tgz'


'/home/wsuser/work/test/seedling_exceptionml.tgz'